In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn import preprocessing
from scipy.io.arff import loadarff 

from mil import MILR
from mil import make_dataset_from_dataframe

import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression

##### Dataset Information
From https://archive.ics.uci.edu/dataset/75/musk+version+2

This dataset describes a set of 102 molecules of which 39 are judged by human experts to be musks and the remaining 63 molecules are judged to be non-musks.  The goal is to learn to predict whether new molecules will be musks or non-musks.  However, the 166 features that describe these molecules depend upon the exact shape, or conformation, of the molecule.  Because bonds can rotate, a single molecule can adopt many different shapes.  To generate this data set, all the low-energy conformations of the molecules were generated to produce 6,598 conformations.  Then, a feature vector was extracted that describes each conformation. 

This many-to-one relationship between feature vectors and molecules is called the "multiple instance problem".  When learning a classifier for this data, the classifier should classify a molecule as "musk" if ANY of its conformations is classified as a musk.  A molecule should be classified as "non-musk" if NONE of its conformations is classified as a musk.


In [3]:
raw_data = loadarff('./data/musk2/musk.arff')
data = pd.DataFrame(raw_data[0])
data.rename(columns={'class': 'target'}, inplace=True)
data.target = data.target.astype(int)

display(data.head(5))
data[['molecule_name', 'target']].value_counts()


,ID,molecule_name,conformation_name,f1,f2,f3,f4,f5,f6,f7,...,f158,f159,f160,f161,f162,f163,f164,f165,f166,target
0,1.0,b'MUSK-211',b'211_1+1',46.0,-108.0,-60.0,-69.0,-117.0,49.0,38.0,...,-308.0,52.0,-7.0,39.0,126.0,156.0,-50.0,-112.0,96.0,1
1,2.0,b'MUSK-211',b'211_1+10',41.0,-188.0,-145.0,22.0,-117.0,-6.0,57.0,...,-59.0,-2.0,52.0,103.0,136.0,169.0,-61.0,-136.0,79.0,1
2,3.0,b'MUSK-211',b'211_1+11',46.0,-194.0,-145.0,28.0,-117.0,73.0,57.0,...,-134.0,-154.0,57.0,143.0,142.0,165.0,-67.0,-145.0,39.0,1
3,4.0,b'MUSK-211',b'211_1+12',41.0,-188.0,-145.0,22.0,-117.0,-7.0,57.0,...,-60.0,-4.0,52.0,104.0,136.0,168.0,-60.0,-135.0,80.0,1
4,5.0,b'MUSK-211',b'211_1+13',41.0,-188.0,-145.0,22.0,-117.0,-7.0,57.0,...,-60.0,-4.0,52.0,104.0,137.0,168.0,-60.0,-135.0,80.0,1


molecule_name     target
b'NON-MUSK-j146'  0         1044
b'NON-MUSK-252'   0         1010
b'NON-MUSK-j147'  0          911
b'NON-MUSK-f146'  0          383
b'NON-MUSK-288'   0          344
                            ... 
b'NON-MUSK-290'   0            2
b'MUSK-300'       1            2
b'MUSK-306'       1            2
b'NON-MUSK-j96'   0            1
b'NON-MUSK-j97'   0            1
Name: count, Length: 102, dtype: int64

In [4]:
feature_names = [f'f{i}' for i in range(1, 167)]
X, y, bags = make_dataset_from_dataframe(data, feature_names, 'target', 'molecule_name')
y_naive = data['target'].values

scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

#### Naive MIL

In [5]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

res = []
y_true = [] # bag level labels
y_pred = [] # bag level predictions
y_prob = [] # bag level probabilities

# iterate over folds of bag level indices
for train, test in kf.split(bags):
    model = LogisticRegression(max_iter=1000)

    # instance level inds for train
    instance_train = np.concatenate([bags[i] for i in train])

    # y_naive is at instance level
    model.fit(X[instance_train], y_naive[instance_train])
    # predict for all the instance so we can use the bag indices to get test bag predictions
    instance_pred = model.predict_proba(X)[:, 1]

    bags_test = [bags[i] for i in test]

    # use the max instance prediction to get bag level prediction
    for bag in bags_test:
        this_y_prob = np.max(instance_pred[bag])
        y_pred.append(this_y_prob > 0.5)
        y_prob.append(this_y_prob)
    
    # y and and test are bag level indices
    y_true.append(y[test])

y_true = np.concatenate(y_true)
y_pred = np.array(y_pred)

this_report = {'bag_fn': 'naive'}
this_report['auc'] = roc_auc_score(y_true, y_prob)
this_report.update(classification_report(y_true, y_pred, output_dict=True))

res = [this_report]

#### Multiple Instance Logistic Regression

In [6]:
for bag_fn in ['max', 'logsumexp', 'product', 'likelihood_ratio']:
    print(bag_fn)
    
    y_true = []
    y_pred = []
    y_prob = []
    for train, test in kf.split(bags):
        bags_train = [bags[i] for i in train]
        bags_test = [bags[i] for i in test]
        model = MILR()
        model.fit(X, y[train], bags_train, epochs=100, lr=1e-2, bag_fn=bag_fn)
        y_true.append(y[test])
        y_pred.append(model.predict(X, bags_test))
        y_prob.append(model.predict_proba(X, bags_test)[:, 1])

    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    y_prob = np.concatenate(y_prob)
    this_report = {'bag_fn': bag_fn}
    this_report['auc'] = roc_auc_score(y_true, y_prob)
    this_report.update(classification_report(y_true, y_pred, output_dict=True))
    res.append(this_report)

# flatten the results
res_flat = []
for this_result in  res:
    this_result_flat = {}
    for k0, v0 in this_result.items():
        if isinstance(v0, dict):
            for k1, v1 in v0.items():
                this_result_flat.update({f'{k0}_{k1}': v1})
        else:
            this_result_flat.update({k0: v0})
    res_flat.append(this_result_flat)

pd.DataFrame(data=res_flat)

    

max
logsumexp
product
likelihood_ratio


,bag_fn,auc,0_precision,0_recall,0_f1-score,0_support,1_precision,1_recall,1_f1-score,1_support,accuracy,macro avg_precision,macro avg_recall,macro avg_f1-score,macro avg_support,weighted avg_precision,weighted avg_recall,weighted avg_f1-score,weighted avg_support
0,naive,0.807489,0.860465,0.587302,0.698113,63.0,0.559322,0.846154,0.673469,39.0,0.686275,0.709894,0.716728,0.685791,102.0,0.745322,0.686275,0.688691,102.0
1,max,0.839235,0.870370,0.746032,0.803419,63.0,0.666667,0.820513,0.735632,39.0,0.774510,0.768519,0.783272,0.769525,102.0,0.792484,0.774510,0.777500,102.0
2,logsumexp,0.871795,0.847458,0.793651,0.819672,63.0,0.697674,0.769231,0.731707,39.0,0.784314,0.772566,0.781441,0.775690,102.0,0.790188,0.784314,0.786039,102.0
3,product,0.777981,0.869565,0.634921,0.733945,63.0,0.589286,0.846154,0.694737,39.0,0.715686,0.729425,0.740537,0.714341,102.0,0.762400,0.715686,0.718954,102.0
4,likelihood_ratio,0.769638,0.883721,0.603175,0.716981,63.0,0.576271,0.871795,0.693878,39.0,0.705882,0.729996,0.737485,0.705429,102.0,0.766167,0.705882,0.708147,102.0
